In [2]:
import os
import pandas as pd
import numpy as np
import re

def process_excel_sheet(folder_path, group_name):
    def process_data(data):
    # Identify datetime columns
        date_columns = data.select_dtypes(include=[np.datetime64]).columns
    
    # Convert datetime columns to string
        data[date_columns] = data[date_columns].apply(lambda x: x.astype(str) if x.name in date_columns else x)
    
        # Create a DataFrame for the sheet name with the same number of columns as data
        sheet_name_df = pd.DataFrame([sheet_name] * len(data.columns)).T
        sheet_name_df.columns = data.columns
        
        # Concatenate sheet name DataFrame with the original data
        data = pd.concat([sheet_name_df, data], ignore_index=True)
        
        # Iterate through the DataFrame rows
        for index, row in data.iterrows():
            for col in range(len(row)):
                # If the cell contains "PRIMARY/SOLD PLAN"
                if row[col] == "PRIMARY/SOLD PLAN":
                    # Fill all cells to the left with the value "PRIMARY/SOLD PLAN"
                    for i in range(col - 1, -1, -1):
                        data.iloc[index, i] = row[col]

        # Extract unique options from the DataFrame
        unique_options = set()
        for index, row in data.iterrows():
            for cell in row:
                if isinstance(cell, str):  # Check if the cell is a string
                    options = re.findall(r'OPTION \d+', cell)  # Extract options using regex
                    unique_options.update(options)

        # Iterate through the DataFrame rows
        for index, row in data.iterrows():
            for col in range(len(row)):
                # If the cell contains any of the unique options
                for option in unique_options:
                    if isinstance(row[col], str) and option in row[col]:
                        # Fill all cells to the left with the corresponding value
                        for i in range(col - 1, -1, -1):
                            data.iloc[index, i] = row[col]
                        break  # Break out of the loop once the option is found

        # Keeping only columns Unnamed: 4 to Unnamed: 7
        df = data.iloc[:, 4:8]

        # Define a function to shift non-NaN values to the left
        def shift_non_nan(row):
            shifted_row = [np.nan] * len(row)
            last_non_nan_index = None
            for i, value in enumerate(row):
                if pd.notnull(value):
                    if last_non_nan_index is None:
                        shifted_row[0] = value
                        last_non_nan_index = 0
                    else:
                        last_non_nan_index += 1
                        shifted_row[last_non_nan_index] = value
            return pd.Series(shifted_row)

        # Apply the function to each row of the dataframe
        df = df.apply(shift_non_nan, axis=1)

        # Find the rows where 'Benefit Plan' is mentioned
        benefit_plan_rows = df[df.apply(lambda row: 'Benefit Plan' in row.values, axis=1)]

        # Merge cells to the right of 'Benefit Plan' rows
        for index, row in benefit_plan_rows.iterrows():
            # Find the index of the column where 'Benefit Plan' is located
            benefit_plan_index = np.where(row == 'Benefit Plan')[0][0]
            # Find non-NaN values to merge
            non_nan_values = row.iloc[benefit_plan_index + 1:].dropna().tolist()
            # Merge non-NaN values into a single cell
            merged_value = ' '.join(str(cell) for cell in non_nan_values)
            # Assign the merged value to the cell next to 'Benefit Plan'
            df.at[index, benefit_plan_index + 1] = merged_value

        # Assuming your DataFrame is named df
        df = df.drop(columns=[2, 3])
        df = df.dropna()

        # Function to split cells containing commas into separate rows for applicable columns
        def split_rows(df):
            new_rows = []
            for index, row in df.iterrows():
                split_row = False  # Flag to check if splitting occurred for this row
                for column_name, cell_value in row.items():
                    if ',' in str(cell_value):
                        split_row = True  # Set flag to True if splitting occurred for any column
                        details = str(cell_value).split(', ')
                        for detail in details:
                            new_row = row.copy()  # Copy the original row
                            new_row[column_name] = detail.strip()  # Update the cell value
                            new_rows.append(new_row)  # Append the new row to the list
                        break  # Break out of the loop once splitting occurs for any column
                if not split_row:
                    new_rows.append(row)  # Append the original row if no splitting occurred
            new_df = pd.DataFrame(new_rows)  # Creating a new DataFrame with split rows
            return new_df

        # Call the function with the DataFrame
        df = split_rows(df)

        # Assign unique indices
        df.index = range(len(df))

        # Words to check for deletion
        words_to_delete = ['OP', 'IP', 'MD', 'X-Ray', 'Lab', 'Riders', 'Drug Benefit', 'Rating Group']

        # Function to filter out rows containing specified words
        def filter_rows(df, words_to_delete):
            indices_to_drop = []  # List to store indices of rows to drop
            # Iterate through each row
            for index, row in df.iterrows():
                # Flag to check if any word to delete is found
                delete_row = False
                # Iterate through each cell value in the row
                for cell_value in row.values:
                    # Check if any word to delete is present in the cell value
                    for word in words_to_delete:
                        if re.search(r'\b' + word + r'\b', str(cell_value)):
                            # If any word is found, set delete_row flag to True
                            delete_row = True
                            break  # No need to continue checking for words in this row
                    if delete_row:
                        break  # No need to check further if any word is found in this row
                if delete_row:
                    indices_to_drop.append(index)  # Store the index of the row to drop
            # Drop the rows outside of the loop to avoid modifying DataFrame while iterating
            df_filtered = df.drop(indices_to_drop)
            return df_filtered

        # Call the function with the DataFrame
        df = filter_rows(df, words_to_delete)

        # Add a space before each $ sign in the DataFrame
        df = df.applymap(lambda x: str(x).replace('$', ' $') if isinstance(x, str) else x)

        # Extracting the words "PCP", "SPC", "UC", "ER", "Ded", and "Coins" into a separate column
        df.insert(loc=1, column='Type', value=df[1].str.extract(r'(PCP|SPC|UC|ER|Ded|Coins|OOP)', expand=False))

        # Fill NaN values in the "Type" column with a default value
        df['Type'].fillna('', inplace=True)

        # Remove the words "PCP", "SPC", "UC", "ER", "Ded", and "Coins" from the actual column
        df[1] = df[1].str.replace(r'(PCP|SPC|UC|ER|Ded|Coins|OOP)', '', regex=True)

        # Merge the first and second columns
        df.insert(loc=0, column='Merged', value=df[0].astype(str) + " " + df['Type'].astype(str))

        # Drop the 'Type' and the original first column
        df.drop(columns=[0, 'Type'], inplace=True)

        # Set column names to empty strings
        df.columns = [''] * len(df.columns)

        # Reset index
        df.reset_index(drop=True, inplace=True)

        # Fill the first NaN value in the first column with 'Group Name'
        df.iloc[:, 0] = df.iloc[:, 0].fillna('Group Name', limit=1)

        # Fill the second NaN value in the first column with 'MNS ID'
        df.iloc[:, 0] = df.iloc[:, 0].fillna('MNS ID', limit=1)

        # Set column names to empty strings
        df.columns = [''] * len(df.columns)

        # Reset index
        df.reset_index(drop=True, inplace=True)

        # Replace words in the DataFrame using the mapping dictionary
        word_mapping = {
            'In Network Copays PCP': 'INN PCP COPAY',
            'In Network Copays SPC': 'INN SPEC COPAY',
            'In Network Copays UC': 'INN URGENT CARE COPAY',
            'In Network Copays ER': 'INN ER COPAY',
            'In Net Ded/Coins/OOP Ded': 'INN DEDUCTIBLE',
            'In Net Ded/Coins/OOP Coins': 'INN COINSURANCE',
            'In Net Ded/Coins/OOP OOP': 'INN OOP Max.',
            'Out of Network Ded': 'OON DEDUCTIBLE',
            'Out of Network Coins': 'OON COINSURANCE',
            'Out of Network OOP': 'OON OOP Max.'
        }

        # Replace words in the DataFrame using the mapping dictionary
        df.replace(word_mapping, inplace=True)

        # Create a new DataFrame to store the result
        new_df = pd.DataFrame(columns=df.columns)

        # Iterate over each row in the original DataFrame
        for index, row in df.iterrows():
            # Check if the row in the second column contains '/'
            if '/' in row[1]:  # Assuming the second column is the one you want to check
                # Split the row into two rows if '/' has '$' before or after it
                parts = row[1].split('/')
                if ('$' in parts[0]) or ('$' in parts[1]):
                    # Add 'IND' prefix to the first split row and 'FAM' to the second split row if necessary
                    new_df.loc[len(new_df)] = ['IND ' + row[0], parts[0]]
                    new_df.loc[len(new_df)] = ['FAM ' + row[0], parts[1]]
                else:
                    new_df.loc[len(new_df)] = [row[0], row[1]]
            else:
                # If no '/', append the row as it is to the new DataFrame
                new_df.loc[len(new_df)] = [row[0], row[1]]

        # Return the processed DataFrame
        return new_df

    
    result_dfs = []  # List to store processed DataFrames

    for file_name in os.listdir(folder_path):
        # Skip files starting with '~$'
        if not file_name.startswith('~$'):
            if file_name.endswith('.xlsx'):
                # Read the Excel file
                file_path = os.path.join(folder_path, file_name)
                try:
                    # Read all sheets from the Excel file
                    sheets = pd.read_excel(file_path, sheet_name=None)
                    # Iterate over each sheet
                    for sheet_name, data in sheets.items():
                        # Check if the sheet_name matches the specified group_name
                        if group_name in sheet_name:
                            # Process the data for the current sheet
                            processed_data = process_data(data)
                            # Replace the first row, first column with "Name"
                            processed_data.iloc[0, 0] = "Name"
                            
                            # Create a new DataFrame containing rows indexed 0 and 1
                            header_df = processed_data.iloc[:2]
                            
                            # Split the DataFrame based on rows starting with "OPTION"
                            split_dfs = []
                            current_df = None
                            for index, row in processed_data.iterrows():
                                if row[0].startswith('OPTION'):
                                    if current_df is not None:
                                        # Concatenate current_df with header_df before appending
                                        split_dfs.append(pd.concat([header_df, current_df], ignore_index=True))
                                    current_df = pd.DataFrame(columns=processed_data.columns)
                                if current_df is not None:
                                    current_df = pd.concat([current_df, row.to_frame().T], ignore_index=True)
                            if current_df is not None:
                                # Concatenate current_df with header_df before appending
                                split_dfs.append(pd.concat([header_df, current_df], ignore_index=True))
                            
                            result_dfs.extend(split_dfs)  # Add split DataFrames to the result list
                            
                except PermissionError as e:
                    print(f"Permission error occurred while accessing {file_name}: {e}")
                except Exception as e:
                    print(f"An error occurred while processing {file_name}: {e}")
    
    return result_dfs

In [5]:

# Define the folder containing the Excel files
folder_path = r"C:\Users\shres\Downloads\Extracted_Tables_Tuftsss"

# Specify the group name you want to process
group_name = "Greater Noida Mental"  # Change this to the desired group name

# Call the function to process the specified group name in the folder
split_dfs = process_excel_sheet(folder_path, group_name)
split_dfs[1]

,,
0,Name,Greater Noida Mental MNS0000192
1,EFFECTIVE DATE,1/1/2024
2,OPTION 2,OPTION 2
3,PRIMARY/SOLD PLAN,PRIMARY/SOLD PLAN
4,Benefit Plan,CPIJ Mod (Granite Advantage EPO OA)
5,IND INN PCP COPAY,$20
6,FAM INN PCP COPAY,$20
7,IND INN SPEC COPAY,$35
8,FAM INN SPEC COPAY,$35
9,INN URGENT CARE COPAY,$20


In [6]:
import os
import pandas as pd
import numpy as np
import re

def process_excel_sheet(folder_path, group_name):
    def process_data(data):
    # Identify datetime columns
        date_columns = data.select_dtypes(include=[np.datetime64]).columns
    
    # Convert datetime columns to string
        data[date_columns] = data[date_columns].apply(lambda x: x.astype(str) if x.name in date_columns else x)
        
        # Extracting sheet name without "MNS"
        filtered_sheet_name = re.sub(r'\bMNS.*', '', sheet_name).strip()

# Create a DataFrame for the filtered sheet name with the same number of columns as data
        filtered_sheet_name_df = pd.DataFrame([filtered_sheet_name] * len(data.columns)).T
        filtered_sheet_name_df.columns = data.columns

# Concatenate filtered sheet name DataFrame with the original data
        data = pd.concat([filtered_sheet_name_df, data], ignore_index=True)
        
        # Iterate through the DataFrame rows
        for index, row in data.iterrows():
            for col in range(len(row)):
                # If the cell contains "PRIMARY/SOLD PLAN"
                if row[col] == "PRIMARY/SOLD PLAN":
                    # Fill all cells to the left with the value "PRIMARY/SOLD PLAN"
                    for i in range(col - 1, -1, -1):
                        data.iloc[index, i] = row[col]

        # Extract unique options from the DataFrame
        unique_options = set()
        for index, row in data.iterrows():
            for cell in row:
                if isinstance(cell, str):  # Check if the cell is a string
                    options = re.findall(r'OPTION \d+', cell)  # Extract options using regex
                    unique_options.update(options)

        # Iterate through the DataFrame rows
        for index, row in data.iterrows():
            for col in range(len(row)):
                # If the cell contains any of the unique options
                for option in unique_options:
                    if isinstance(row[col], str) and option in row[col]:
                        # Fill all cells to the left with the corresponding value
                        for i in range(col - 1, -1, -1):
                            data.iloc[index, i] = row[col]
                        break  # Break out of the loop once the option is found

        # Keeping only columns Unnamed: 4 to Unnamed: 7
        df = data.iloc[:, 4:8]

        # Define a function to shift non-NaN values to the left
        def shift_non_nan(row):
            shifted_row = [np.nan] * len(row)
            last_non_nan_index = None
            for i, value in enumerate(row):
                if pd.notnull(value):
                    if last_non_nan_index is None:
                        shifted_row[0] = value
                        last_non_nan_index = 0
                    else:
                        last_non_nan_index += 1
                        shifted_row[last_non_nan_index] = value
            return pd.Series(shifted_row)

        # Apply the function to each row of the dataframe
        df = df.apply(shift_non_nan, axis=1)

        # Find the rows where 'Benefit Plan' is mentioned
        benefit_plan_rows = df[df.apply(lambda row: 'Benefit Plan' in row.values, axis=1)]

        # Merge cells to the right of 'Benefit Plan' rows
        for index, row in benefit_plan_rows.iterrows():
            # Find the index of the column where 'Benefit Plan' is located
            benefit_plan_index = np.where(row == 'Benefit Plan')[0][0]
            # Find non-NaN values to merge
            non_nan_values = row.iloc[benefit_plan_index + 1:].dropna().tolist()
            # Merge non-NaN values into a single cell
            merged_value = ' '.join(str(cell) for cell in non_nan_values)
            # Assign the merged value to the cell next to 'Benefit Plan'
            df.at[index, benefit_plan_index + 1] = merged_value

        # Assuming your DataFrame is named df
        df = df.drop(columns=[2, 3])
        df = df.dropna()

        # Function to split cells containing commas into separate rows for applicable columns
        def split_rows(df):
            new_rows = []
            for index, row in df.iterrows():
                split_row = False  # Flag to check if splitting occurred for this row
                for column_name, cell_value in row.items():
                    if ',' in str(cell_value):
                        split_row = True  # Set flag to True if splitting occurred for any column
                        details = str(cell_value).split(', ')
                        for detail in details:
                            new_row = row.copy()  # Copy the original row
                            new_row[column_name] = detail.strip()  # Update the cell value
                            new_rows.append(new_row)  # Append the new row to the list
                        break  # Break out of the loop once splitting occurs for any column
                if not split_row:
                    new_rows.append(row)  # Append the original row if no splitting occurred
            new_df = pd.DataFrame(new_rows)  # Creating a new DataFrame with split rows
            return new_df

        # Call the function with the DataFrame
        df = split_rows(df)

        # Assign unique indices
        df.index = range(len(df))

        # Words to check for deletion
        words_to_delete = ['OP', 'IP', 'MD', 'X-Ray', 'Lab', 'Riders', 'Drug Benefit', 'Rating Group']

        # Function to filter out rows containing specified words
        def filter_rows(df, words_to_delete):
            indices_to_drop = []  # List to store indices of rows to drop
            # Iterate through each row
            for index, row in df.iterrows():
                # Flag to check if any word to delete is found
                delete_row = False
                # Iterate through each cell value in the row
                for cell_value in row.values:
                    # Check if any word to delete is present in the cell value
                    for word in words_to_delete:
                        if re.search(r'\b' + word + r'\b', str(cell_value)):
                            # If any word is found, set delete_row flag to True
                            delete_row = True
                            break  # No need to continue checking for words in this row
                    if delete_row:
                        break  # No need to check further if any word is found in this row
                if delete_row:
                    indices_to_drop.append(index)  # Store the index of the row to drop
            # Drop the rows outside of the loop to avoid modifying DataFrame while iterating
            df_filtered = df.drop(indices_to_drop)
            return df_filtered

        # Call the function with the DataFrame
        df = filter_rows(df, words_to_delete)

        # Add a space before each $ sign in the DataFrame
        df = df.applymap(lambda x: str(x).replace('$', ' $') if isinstance(x, str) else x)

        # Extracting the words "PCP", "SPC", "UC", "ER", "Ded", and "Coins" into a separate column
        df.insert(loc=1, column='Type', value=df[1].str.extract(r'(PCP|SPC|UC|ER|Ded|Coins|OOP)', expand=False))

        # Fill NaN values in the "Type" column with a default value
        df['Type'].fillna('', inplace=True)

        # Remove the words "PCP", "SPC", "UC", "ER", "Ded", and "Coins" from the actual column
        df[1] = df[1].str.replace(r'(PCP|SPC|UC|ER|Ded|Coins|OOP)', '', regex=True)

        # Merge the first and second columns
        df.insert(loc=0, column='Merged', value=df[0].astype(str) + " " + df['Type'].astype(str))

        # Drop the 'Type' and the original first column
        df.drop(columns=[0, 'Type'], inplace=True)

        # Set column names to empty strings
        df.columns = [''] * len(df.columns)

        # Reset index
        df.reset_index(drop=True, inplace=True)

        # Fill the first NaN value in the first column with 'Group Name'
        df.iloc[:, 0] = df.iloc[:, 0].fillna('Group Name', limit=1)

        # Fill the second NaN value in the first column with 'MNS ID'
        df.iloc[:, 0] = df.iloc[:, 0].fillna('MNS ID', limit=1)

        # Set column names to empty strings
        df.columns = [''] * len(df.columns)

        # Reset index
        df.reset_index(drop=True, inplace=True)

        # Replace words in the DataFrame using the mapping dictionary
        word_mapping = {
            'In Network Copays PCP': 'INN PCP COPAY',
            'In Network Copays SPC': 'INN SPEC COPAY',
            'In Network Copays UC': 'INN URGENT CARE COPAY',
            'In Network Copays ER': 'INN ER COPAY',
            'In Net Ded/Coins/OOP Ded': 'INN DEDUCTIBLE',
            'In Net Ded/Coins/OOP Coins': 'INN COINSURANCE',
            'In Net Ded/Coins/OOP OOP': 'INN OOP Max.',
            'Out of Network Ded': 'OON DEDUCTIBLE',
            'Out of Network Coins': 'OON COINSURANCE',
            'Out of Network OOP': 'OON OOP Max.'
        }

        # Replace words in the DataFrame using the mapping dictionary
        df.replace(word_mapping, inplace=True)

        # Create a new DataFrame to store the result
        new_df = pd.DataFrame(columns=df.columns)

        # Iterate over each row in the original DataFrame
        for index, row in df.iterrows():
            # Check if the row in the second column contains '/'
            if '/' in row[1]:  # Assuming the second column is the one you want to check
                # Split the row into two rows if '/' has '$' before or after it
                parts = row[1].split('/')
                if ('$' in parts[0]) or ('$' in parts[1]):
                    # Add 'IND' prefix to the first split row and 'FAM' to the second split row if necessary
                    new_df.loc[len(new_df)] = ['IND ' + row[0], parts[0]]
                    new_df.loc[len(new_df)] = ['FAM ' + row[0], parts[1]]
                else:
                    new_df.loc[len(new_df)] = [row[0], row[1]]
            else:
                # If no '/', append the row as it is to the new DataFrame
                new_df.loc[len(new_df)] = [row[0], row[1]]

        # Return the processed DataFrame
        return new_df

    
    result_dfs = []  # List to store processed DataFrames

    for file_name in os.listdir(folder_path):
        # Skip files starting with '~$'
        if not file_name.startswith('~$'):
            if file_name.endswith('.xlsx'):
                # Read the Excel file
                file_path = os.path.join(folder_path, file_name)
                try:
                    # Read all sheets from the Excel file
                    sheets = pd.read_excel(file_path, sheet_name=None)
                    # Iterate over each sheet
                    for sheet_name, data in sheets.items():
                        # Check if the sheet_name matches the specified group_name
                        if group_name in sheet_name:
                            # Process the data for the current sheet
                            processed_data = process_data(data)
                            # Replace the first row, first column with "Name"
                            processed_data.iloc[0, 0] = "Name"
                            
                            # Create a new DataFrame containing rows indexed 0 and 1
                            header_df = processed_data.iloc[:2]
                            
                            # Split the DataFrame based on rows starting with "OPTION"
                            split_dfs = []
                            current_df = None
                            for index, row in processed_data.iterrows():
                                if row[0].startswith('OPTION'):
                                    if current_df is not None:
                                        # Concatenate current_df with header_df before appending
                                        split_dfs.append(pd.concat([header_df, current_df], ignore_index=True))
                                    current_df = pd.DataFrame(columns=processed_data.columns)
                                if current_df is not None:
                                    current_df = pd.concat([current_df, row.to_frame().T], ignore_index=True)
                            if current_df is not None:
                                # Concatenate current_df with header_df before appending
                                split_dfs.append(pd.concat([header_df, current_df], ignore_index=True))
                            
                            result_dfs.extend(split_dfs)  # Add split DataFrames to the result list
                            
                except PermissionError as e:
                    print(f"Permission error occurred while accessing {file_name}: {e}")
                except Exception as e:
                    print(f"An error occurred while processing {file_name}: {e}")
    
    return result_dfs

In [9]:

# Define the folder containing the Excel files
folder_path = r"C:\Users\shres\Downloads\Extracted_Tables_Tuftsss"

# Specify the group name you want to process
group_name = "Greater Noida Mental"  # Change this to the desired group name

# Call the function to process the specified group name in the folder
split_dfs = process_excel_sheet(folder_path, group_name)
split_dfs

[                                                              
 0                    Name                 Greater Noida Mental
 1         EFFECTIVE DATE                              1/1/2024
 2               OPTION 1                              OPTION 1
 3      PRIMARY/SOLD PLAN                     PRIMARY/SOLD PLAN
 4           Benefit Plan   CPIJ Mod (Granite Advantage EPO OA)
 5       IND INN PCP COPAY                                  $10
 6       FAM INN PCP COPAY                                  $10
 7      IND INN SPEC COPAY                                  $25
 8      FAM INN SPEC COPAY                                  $25
 9   INN URGENT CARE COPAY                                  $10
 10           INN ER COPAY                                 $230
 11     IND INN DEDUCTIBLE                               $5,000
 12     FAM INN DEDUCTIBLE                        $10,000 (Emb)
 13        INN COINSURANCE                                  80%
 14       IND INN OOP Max.              